In [13]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [14]:
csv = pd.read_csv('../../data/notification_sequence.csv', sep = ',')

In [16]:
new_csv = {
    'gender': [],
    'age': [],
    'department': [],
    'scenario': [],
    'view_order': [],
    'total': [],
    'social': [],
    'communication': [],
    'entertainment': [],
    'news': [],
    'system': [],
    'notification': []
}
Y_111111 = []
Y_1122 = []
Y_123 = []
Y_222 = []
Y_24 = []
Y_15 = []

for idx, row in csv.iterrows():
    if row['social_usetime'] == '?':
        continue
    tot = int(row['social_usetime']) + int(row['communication_usetime']) + int(row['entertainment_usetime']) + int(row['news_usetime']) + int(row['system_usetime']) + int(row['notification_usetime'])
    if tot == 0:
        continue
    new_csv['gender'].append(row['gender'])
    new_csv['age'].append(row['age'])
    new_csv['department'].append(row['department'])
    new_csv['scenario'].append(row['scenario'])
    new_csv['view_order'].append(1 if row['view_order_top'] == 1 else (0 if row['view_order_middle'] == 1 else -1))
    new_csv['total'].append(row['total_usetime'])
    new_csv['social'].append(int(row['social_usetime']) / tot)
    new_csv['communication'].append(int(row['communication_usetime']) / tot)
    new_csv['entertainment'].append(int(row['entertainment_usetime']) / tot)
    new_csv['news'].append(int(row['news_usetime']) / tot)
    new_csv['system'].append(int(row['system_usetime']) / tot)
    new_csv['notification'].append(int(row['notification_usetime']) / tot)
    Y_111111.append(row['tol_111111'])
    Y_1122.append(row['tol_1122'])
    Y_123.append(row['tol_123'])
    Y_222.append(row['tol_222'])
    Y_24.append(row['tol_24'])
    Y_15.append(row['tol_15'])

new_csv = pd.get_dummies(pd.DataFrame(new_csv))
X = np.array(new_csv.values)

In [17]:
for tol, tol_name in [(Y_111111, '111111'), (Y_1122, '1122'), (Y_123, '123'), (Y_222, '222'), (Y_24, '24'), (Y_15, '15')]:
    accu = []
    for i in range(100):
        X_train, X_test, Y_train, Y_test = train_test_split(X, tol, train_size=0.7)

        _mean = np.mean(X_train, axis=0)
        _std = np.std(X_train, axis=0)
        _scale = np.full(X_train.shape[1], 1.0)
        _scale[20:24] *= 0.25
        _scale[3:6] *= 2.85

        X_train -= _mean
        X_train /= _std
        X_train *= _scale
        X_test -= _mean
        X_test /= _std
        X_test *= _scale

        rand_forest = RandomForestClassifier(n_estimators=15, max_depth=8, min_samples_leaf=5)
        rand_forest.fit(X_train, Y_train)
        Y_pred = rand_forest.predict(X_test)
        accu.append(accuracy_score(Y_test, Y_pred))
    print('Random Forest Tolerance-%s Accuracy=%.3f%%' % (tol_name, np.mean(accu) * 100))

Random Forest Tolerance-111111 Accuracy=14.093%
Random Forest Tolerance-1122 Accuracy=20.418%
Random Forest Tolerance-123 Accuracy=27.397%
Random Forest Tolerance-222 Accuracy=25.038%
Random Forest Tolerance-24 Accuracy=48.506%
Random Forest Tolerance-15 Accuracy=55.852%
